In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from htools import FuzzyKeyDict

In [3]:
!pwd

/Users/hmamin/pythonhm/htools/notebooks


In [5]:
d = dict(dog=3, bulldog=2, doggo=4, housecat=0, kitty=-3, kitten=-4, horse=99,
         racehorse=101)
fd = FuzzyKeyDict(d)
d, fd

({'dog': 3,
  'bulldog': 2,
  'doggo': 4,
  'housecat': 0,
  'kitty': -3,
  'kitten': -4,
  'horse': 99,
  'racehorse': 101},
 {'dog': 3,
  'bulldog': 2,
  'doggo': 4,
  'housecat': 0,
  'kitty': -3,
  'kitten': -4,
  'horse': 99,
  'racehorse': 101})

In [6]:
fd['dog']

3

In [7]:
fd['doggy']

[4, 3, 2]

In [39]:
class LowerDict(dict):
    
    def __init__(self, dummy, data=()):
        super().__init__(data)
        self.dummy = dummy
        
    def __getitem__(self, key):
        if key in self:
            return super().__getitem__(key)
        return [super().__getitem__(key.lower())]*3

In [40]:
ld = LowerDict(6, d)
ld

{'dog': 3,
 'bulldog': 2,
 'doggo': 4,
 'housecat': 0,
 'kitty': -3,
 'kitten': -4,
 'horse': 99,
 'racehorse': 101}

In [41]:
ld['dog']

3

In [42]:
ld['DOG']

[3, 3, 3]

In [28]:
ld.dummy

6

In [29]:
ld.pop('doggo')

4

In [30]:
ld

{'dog': 3,
 'bulldog': 2,
 'housecat': 0,
 'kitty': -3,
 'kitten': -4,
 'horse': 99,
 'racehorse': 101}

In [31]:
d

{'dog': 3,
 'bulldog': 2,
 'doggo': 4,
 'housecat': 0,
 'kitty': -3,
 'kitten': -4,
 'horse': 99,
 'racehorse': 101}

In [32]:
'dog' in ld, 'cow' in ld

(True, False)

In [1]:
from collections import namedtuple, Iterable, Mapping, UserDict
from fuzzywuzzy import fuzz, process
import numpy as np

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


In [40]:
class FuzzyKeyDict(dict):
    """Dictionary that will try to find similar keys if a key is missing and
    return their corresponding values. This can be useful when working with
    embeddings: for example, we can try mapping missing words to a combination
    of existing words.

    Examples
    --------
    d = FuzzyKeyDict(limit=3)
    d['dog'] = 0
    d['cat'] = 1
    d['alley cat'] = 2
    d['pig'] = 3
    d['cow'] = 4
    d['cowbell'] = 5
    d['baby cow'] = 6

    # Keys and similarity scores for the most similar keys.
    >>> d.similar_keys('house cat')
    [('alley cat', 56), ('cat', 50), ('cowbell', 25)]

    # "house cat" not in dict so we get the values for the most similar keys.
    >>> d['house cat']
    [2, 1, 5]

    # "cat" is in dict so output is an integer rather than a list.
    >>> d['cat']
    1
    """

    def __init__(self, data=(), limit=3, return_list=False):
        """
        Parameters
        ----------
        data: Iterable (optional)
            Sequence of pairs, such as a dictionary or a list of tuples. If
            provided, this will be used to populate the FuzzyKeyDict.
        limit: int
            Number of similar keys to find when trying to retrieve the value
            for a missing key.
        return_list: bool
            If True, __getitem__ will always return a list of len `limit`. If
            False, it will return a key's corresponding value if it's present
            and a list of values for the `limit` closest keys if it's not.
        """
        super().__init__(data)
        self.limit = limit
        self.return_list = return_list

    def __getitem__(self, key):
        """
        Returns
        -------
        any or list[any]: If key is present in dict, the corresponding value
            is returned. If not, the n closest keys are identified and their
            corresponding values are returned in a list (where n is defined
            by the `limit` argument specified in the constructor). Values are
            sorted in descending order by the neighboring keys' similarity to
            the missing key in.
        """
        try:
            res = super().__getitem__(key)
            return [res]*self.limit if self.return_list else res
        except KeyError:
            return [dict.__getitem__(self, k) for k in self.similar_keys(key)]

    def similar_keys(self, key, return_similarities=False):
        pairs = process.extract(key, self.keys(), limit=self.limit,
                                scorer=fuzz.ratio)
        if return_similarities:
            return pairs
        return [p[0] for p in pairs]

In [8]:
d = dict(dog=3, bulldog=2, doggo=4, housecat=0, kitty=-3, kitten=-4, horse=99,
         racehorse=101)
fd = FuzzyKeyDict(d)
fd

{'dog': 3,
 'bulldog': 2,
 'doggo': 4,
 'housecat': 0,
 'kitty': -3,
 'kitten': -4,
 'horse': 99,
 'racehorse': 101}

In [9]:
fd['dog']

3

In [10]:
isinstance(fd, FuzzyKeyDict)

True

In [11]:
fd['wolf']

[3, 4, 99]

In [12]:
fd = FuzzyKeyDict(d, return_list=True)
fd

{'dog': [3, 3, 3],
 'bulldog': [2, 2, 2],
 'doggo': [4, 4, 4],
 'housecat': [0, 0, 0],
 'kitty': [-3, -3, -3],
 'kitten': [-4, -4, -4],
 'horse': [99, 99, 99],
 'racehorse': [101, 101, 101]}

In [13]:
fd['dog']

[3, 3, 3]

In [14]:
fd['wolf']

[3, 4, 99]

In [15]:
fd['cat'] = 44

In [16]:
fd

{'dog': [3, 3, 3],
 'bulldog': [2, 2, 2],
 'doggo': [4, 4, 4],
 'housecat': [0, 0, 0],
 'kitty': [-3, -3, -3],
 'kitten': [-4, -4, -4],
 'horse': [99, 99, 99],
 'racehorse': [101, 101, 101],
 'cat': [44, 44, 44]}

In [17]:
fd['cat']

[44, 44, 44]

In [18]:
d = dict(zip(['housecat', 'kitty', 'cat', 'dog', 'bulldog', 'frog', 'bullfrog'], 
             np.random.randn(7, 3)))
d

{'housecat': array([ 0.93608848,  1.70322904, -1.44291479]),
 'kitty': array([-0.5779825 , -1.2137863 ,  1.38845792]),
 'cat': array([ 0.87151834, -0.78733413,  0.4918777 ]),
 'dog': array([ 0.80513048, -0.52123506,  0.66851821]),
 'bulldog': array([0.90517271, 0.6464598 , 0.51729801]),
 'frog': array([-2.01104731, -0.98391542, -1.11886689]),
 'bullfrog': array([1.12202647, 1.04335869, 0.18002501])}

In [19]:
fd = FuzzyKeyDict(d)
fd

{'housecat': array([ 0.93608848,  1.70322904, -1.44291479]),
 'kitty': array([-0.5779825 , -1.2137863 ,  1.38845792]),
 'cat': array([ 0.87151834, -0.78733413,  0.4918777 ]),
 'dog': array([ 0.80513048, -0.52123506,  0.66851821]),
 'bulldog': array([0.90517271, 0.6464598 , 0.51729801]),
 'frog': array([-2.01104731, -0.98391542, -1.11886689]),
 'bullfrog': array([1.12202647, 1.04335869, 0.18002501])}

In [20]:
fd['housecat']

array([ 0.93608848,  1.70322904, -1.44291479])

In [21]:
fd['pussycat']

[array([ 0.93608848,  1.70322904, -1.44291479]),
 array([ 0.87151834, -0.78733413,  0.4918777 ]),
 array([-0.5779825 , -1.2137863 ,  1.38845792])]

In [30]:
fd = FuzzyKeyDict(d, 4, return_list=True)
fd

{'housecat': [array([ 0.93608848,  1.70322904, -1.44291479]),
  array([ 0.93608848,  1.70322904, -1.44291479]),
  array([ 0.93608848,  1.70322904, -1.44291479]),
  array([ 0.93608848,  1.70322904, -1.44291479])],
 'kitty': [array([-0.5779825 , -1.2137863 ,  1.38845792]),
  array([-0.5779825 , -1.2137863 ,  1.38845792]),
  array([-0.5779825 , -1.2137863 ,  1.38845792]),
  array([-0.5779825 , -1.2137863 ,  1.38845792])],
 'cat': [array([ 0.87151834, -0.78733413,  0.4918777 ]),
  array([ 0.87151834, -0.78733413,  0.4918777 ]),
  array([ 0.87151834, -0.78733413,  0.4918777 ]),
  array([ 0.87151834, -0.78733413,  0.4918777 ])],
 'dog': [array([ 0.80513048, -0.52123506,  0.66851821]),
  array([ 0.80513048, -0.52123506,  0.66851821]),
  array([ 0.80513048, -0.52123506,  0.66851821]),
  array([ 0.80513048, -0.52123506,  0.66851821])],
 'bulldog': [array([0.90517271, 0.6464598 , 0.51729801]),
  array([0.90517271, 0.6464598 , 0.51729801]),
  array([0.90517271, 0.6464598 , 0.51729801]),
  array([

In [31]:
fd['cat']

[array([ 0.87151834, -0.78733413,  0.4918777 ]),
 array([ 0.87151834, -0.78733413,  0.4918777 ]),
 array([ 0.87151834, -0.78733413,  0.4918777 ]),
 array([ 0.87151834, -0.78733413,  0.4918777 ])]

In [32]:
fd['mule']

[array([0.90517271, 0.6464598 , 0.51729801]),
 array([ 0.93608848,  1.70322904, -1.44291479]),
 array([1.12202647, 1.04335869, 0.18002501]),
 array([-0.5779825 , -1.2137863 ,  1.38845792])]

In [33]:
fd.get('mule')

In [34]:
fd.get('cat')

array([ 0.87151834, -0.78733413,  0.4918777 ])

In [35]:
np.mean(fd['duck'], 0)

array([ 0.5086887 , -0.20478161,  0.27648476])

In [36]:
np.mean(fd['duck'], 1)

array([ 0.39880091,  0.19202064,  0.31747121, -0.13443696])

In [37]:
fd['duck']

[array([ 0.93608848,  1.70322904, -1.44291479]),
 array([ 0.87151834, -0.78733413,  0.4918777 ]),
 array([ 0.80513048, -0.52123506,  0.66851821]),
 array([-0.5779825 , -1.2137863 ,  1.38845792])]

In [39]:
fd.similar_keys('cow'), fd.similar_keys('cow', True)

(['cat', 'dog', 'frog', 'bulldog'],
 [('cat', 33), ('dog', 33), ('frog', 29), ('bulldog', 20)])